# Image retrieval using deep features

In [1]:
import graphlab

# Load some CIFAR-10 images

In [2]:
image_train_data = graphlab.SFrame('image_train_data')

[INFO] graphlab.cython.cy_server: GraphLab Create v2.1 started. Logging: /tmp/graphlab_server_1588583571.log


This non-commercial license of GraphLab Create for academic use is assigned to piyushchohan48@gmail.com and will expire on May 01, 2021.


In [3]:
image_train_data

id,image,label,deep_features,image_array
24,Height: 32 Width: 32,bird,"[0.242871761322,1.09545373917, 0.0, ...","[73.0, 77.0, 58.0, 71.0,68.0, 50.0, 77.0, 69.0, ..."
33,Height: 32 Width: 32,cat,"[0.525087952614, 0.0,0.0, 0.0, 0.0, 0.0, ...","[7.0, 5.0, 8.0, 7.0, 5.0,8.0, 5.0, 4.0, 6.0, 7.0, ..."
36,Height: 32 Width: 32,cat,"[0.566015958786, 0.0,0.0, 0.0, 0.0, 0.0, ...","[169.0, 122.0, 65.0,131.0, 108.0, 75.0, ..."
70,Height: 32 Width: 32,dog,"[1.12979578972, 0.0, 0.0,0.778194487095, 0.0, ...","[154.0, 179.0, 152.0,159.0, 183.0, 157.0, ..."
90,Height: 32 Width: 32,bird,"[1.71786928177, 0.0, 0.0,0.0, 0.0, 0.0, ...","[216.0, 195.0, 180.0,201.0, 178.0, 160.0, ..."
97,Height: 32 Width: 32,automobile,"[1.57818555832, 0.0, 0.0,0.0, 0.0, 0.0, ...","[33.0, 44.0, 27.0, 29.0,44.0, 31.0, 32.0, 45.0, ..."
107,Height: 32 Width: 32,dog,"[0.0, 0.0,0.220677852631, 0.0, ...","[97.0, 51.0, 31.0, 104.0,58.0, 38.0, 107.0, 61.0, ..."
121,Height: 32 Width: 32,bird,"[0.0, 0.23753464222, 0.0,0.0, 0.0, 0.0, ...","[93.0, 96.0, 88.0, 102.0,106.0, 97.0, 117.0, ..."
136,Height: 32 Width: 32,automobile,"[0.0, 0.0, 0.0, 0.0, 0.0,0.0, 7.5737862587, 0.0, ...","[35.0, 59.0, 53.0, 36.0,56.0, 56.0, 42.0, 62.0, ..."
138,Height: 32 Width: 32,bird,"[0.658935725689, 0.0,0.0, 0.0, 0.0, 0.0, ...","[205.0, 193.0, 195.0,200.0, 187.0, 193.0, ..."


# Compute deep features from our images

In [4]:
#deep_learning_model = graphlab.load_model('imagenet_model_iter45')
#image_data['deep_features'] = deep_learning_model.extract_features(image_data)

# Create a nearest neighbors model to retrieve images from deep features

In [5]:
knn_model = graphlab.nearest_neighbors.create(image_train_data,
                                               features = ['deep_features'],
                                               label = 'id')

Starting brute force nearest neighbors model training.

# Use image retrieval model with deep features to find similar images

In [6]:
cat = image_train_data[18:19]

In [7]:
graphlab.canvas.set_target('ipynb')
cat['image'].show()

In [8]:
knn_model.query(cat)

Starting pairwise querying.

+--------------+---------+-------------+--------------+

| Query points | # Pairs | % Complete. | Elapsed Time |

+--------------+---------+-------------+--------------+

| 0            | 1       | 0.0498753   | 121.335ms    |

| Done         |         | 100         | 563.48ms     |

+--------------+---------+-------------+--------------+

query_label,reference_label,distance,rank
0,384,0.0,1
0,6910,36.9403137951,2
0,39777,38.4634888975,3
0,36870,39.7559623119,4
0,41734,39.7866014148,5


In [9]:
def get_images_from_ids(query_result):
    return image_train_data.filter_by(query_result['reference_label'],'id')

In [10]:
cat_neighbors = get_images_from_ids(knn_model.query(cat))

Starting pairwise querying.

+--------------+---------+-------------+--------------+

| Query points | # Pairs | % Complete. | Elapsed Time |

+--------------+---------+-------------+--------------+

| 0            | 1       | 0.0498753   | 33.34ms      |

| Done         |         | 100         | 421.363ms    |

+--------------+---------+-------------+--------------+

In [11]:
cat_neighbors['image'].show()

# Find images similar to a car

In [12]:
car = image_train_data[8:9]

In [13]:
car['image'].show()

In [14]:
get_images_from_ids(knn_model.query(car))['image'].show()

Starting pairwise querying.

+--------------+---------+-------------+--------------+

| Query points | # Pairs | % Complete. | Elapsed Time |

+--------------+---------+-------------+--------------+

| 0            | 1       | 0.0498753   | 38.811ms     |

| Done         |         | 100         | 507.722ms    |

+--------------+---------+-------------+--------------+

# Create a lambda function to find and show nearest neighbors to an image

In [15]:
show_neighbors = lambda i: get_images_from_ids(knn_model.query(image_train_data[i:i+1]))['image'].show()

In [16]:
show_neighbors(8)

Starting pairwise querying.

+--------------+---------+-------------+--------------+

| Query points | # Pairs | % Complete. | Elapsed Time |

+--------------+---------+-------------+--------------+

| 0            | 1       | 0.0498753   | 19.683ms     |

| Done         |         | 100         | 438.939ms    |

+--------------+---------+-------------+--------------+

In [17]:
show_neighbors(26)

Starting pairwise querying.

+--------------+---------+-------------+--------------+

| Query points | # Pairs | % Complete. | Elapsed Time |

+--------------+---------+-------------+--------------+

| 0            | 1       | 0.0498753   | 38.93ms      |

| Done         |         | 100         | 357.24ms     |

+--------------+---------+-------------+--------------+

In [18]:
show_neighbors(500)

Starting pairwise querying.

+--------------+---------+-------------+--------------+

| Query points | # Pairs | % Complete. | Elapsed Time |

+--------------+---------+-------------+--------------+

| 0            | 1       | 0.0498753   | 39.791ms     |

| Done         |         | 100         | 442.265ms    |

+--------------+---------+-------------+--------------+

In [19]:
show_neighbors(100)

Starting pairwise querying.

+--------------+---------+-------------+--------------+

| Query points | # Pairs | % Complete. | Elapsed Time |

+--------------+---------+-------------+--------------+

| 0            | 1       | 0.0498753   | 16.077ms     |

| Done         |         | 100         | 713.405ms    |

+--------------+---------+-------------+--------------+

In [20]:
len(image_train_data)

2005

In [21]:
len(image_train_data)

2005

# Week-6 Assignment Begins

In [22]:
sketch = graphlab.Sketch(image_train_data['label'])

In [23]:
sketch


+------------------+-------+----------+
|       item       | value | is exact |
+------------------+-------+----------+
|      Length      |  2005 |   Yes    |
| # Missing Values |   0   |   Yes    |
| # unique values  |   4   |    No    |
+------------------+-------+----------+

Most frequent items:
+-------+------------+-----+-----+------+
| value | automobile | cat | dog | bird |
+-------+------------+-----+-----+------+
| count |    509     | 509 | 509 | 478  |
+-------+------------+-----+-----+------+


In [24]:
# The least common category is BIRD

In [25]:
dog_data=image_train_data[image_train_data['label']=='dog']

In [26]:
dog_data

id,image,label,deep_features,image_array
70,Height: 32 Width: 32,dog,"[1.12979578972, 0.0, 0.0,0.778194487095, 0.0, ...","[154.0, 179.0, 152.0,159.0, 183.0, 157.0, ..."
107,Height: 32 Width: 32,dog,"[0.0, 0.0,0.220677852631, 0.0, ...","[97.0, 51.0, 31.0, 104.0,58.0, 38.0, 107.0, 61.0, ..."
177,Height: 32 Width: 32,dog,"[0.0, 1.45965671539, 0.0,0.422992348671, 0.0, ...","[55.0, 75.0, 42.0, 51.0,76.0, 37.0, 57.0, 83.0, ..."
424,Height: 32 Width: 32,dog,"[0.942399680614, 0.0,0.220352768898, 0.0, ...","[60.0, 35.0, 18.0, 63.0,49.0, 38.0, 66.0, 56.0, ..."
462,Height: 32 Width: 32,dog,"[1.43462562561, 0.0, 0.0,0.0, 0.0, 0.0, ...","[86.0, 69.0, 75.0, 57.0,41.0, 48.0, 46.0, 35.0, ..."
542,Height: 32 Width: 32,dog,"[0.451547086239, 0.0,0.0, 0.0, 0.0, 0.0, ...","[196.0, 174.0, 113.0,140.0, 117.0, 65.0, 8 ..."
573,Height: 32 Width: 32,dog,"[0.592360973358, 0.0,0.0, 0.0, 0.0, 0.0, ...","[153.0, 103.0, 52.0,151.0, 102.0, 49.0, ..."
851,Height: 32 Width: 32,dog,"[0.690123438835, 0.0,0.0, 0.0, 0.305860161 ...","[39.0, 6.0, 4.0, 53.0,23.0, 24.0, 57.0, 37.0, ..."
919,Height: 32 Width: 32,dog,"[0.0, 0.177558660507,0.139396846294, 0.0, ...","[29.0, 43.0, 4.0, 24.0,35.0, 6.0, 24.0, 37.0, ..."
1172,Height: 32 Width: 32,dog,"[0.517601490021, 0.0,1.96418333054, 0.0, 0.0, ...","[182.0, 180.0, 197.0,196.0, 192.0, 209.0, ..."


In [27]:
len(dog_data)

509

In [28]:
cat_data=image_train_data[image_train_data['label']=='cat']

In [29]:
cat_data

id,image,label,deep_features,image_array
33,Height: 32 Width: 32,cat,"[0.525087952614, 0.0,0.0, 0.0, 0.0, 0.0, ...","[7.0, 5.0, 8.0, 7.0, 5.0,8.0, 5.0, 4.0, 6.0, 7.0, ..."
36,Height: 32 Width: 32,cat,"[0.566015958786, 0.0,0.0, 0.0, 0.0, 0.0, ...","[169.0, 122.0, 65.0,131.0, 108.0, 75.0, ..."
159,Height: 32 Width: 32,cat,"[0.0, 0.0, 0.0,0.64327532053, 0.0, 0.0, ...","[154.0, 145.0, 135.0,152.0, 144.0, 135.0, ..."
331,Height: 32 Width: 32,cat,"[0.0, 0.0,0.510963916779, 0.0, ...","[45.0, 65.0, 92.0, 72.0,95.0, 110.0, 106.0, ..."
367,Height: 32 Width: 32,cat,"[1.38658058643, 0.0, 0.0,0.0, 0.0, 0.182891070 ...","[168.0, 151.0, 143.0,145.0, 130.0, 124.0, ..."
384,Height: 32 Width: 32,cat,"[1.04403531551, 0.0, 0.0,0.0, 0.0, 0.0, ...","[46.0, 45.0, 50.0, 47.0,45.0, 51.0, 45.0, 44.0, ..."
494,Height: 32 Width: 32,cat,"[0.0, 0.0539512038231,1.95745122433, 0.0, 0.0, ...","[26.0, 34.0, 29.0, 24.0,29.0, 25.0, 33.0, 43.0, ..."
597,Height: 32 Width: 32,cat,"[0.0, 0.0,0.0470637083054, 0.0, ...","[133.0, 153.0, 138.0,126.0, 146.0, 136.0, ..."
788,Height: 32 Width: 32,cat,"[0.505841910839, 0.0,0.0, 0.0, 0.427211523 ...","[184.0, 200.0, 197.0,189.0, 203.0, 200.0, ..."
882,Height: 32 Width: 32,cat,"[0.0, 0.0,0.156200289726, 0.0, ...","[141.0, 133.0, 112.0,143.0, 133.0, 113.0, ..."


In [30]:
bird_data=image_train_data[image_train_data['label']=='bird']

In [31]:
bird_data

id,image,label,deep_features,image_array
24,Height: 32 Width: 32,bird,"[0.242871761322,1.09545373917, 0.0, ...","[73.0, 77.0, 58.0, 71.0,68.0, 50.0, 77.0, 69.0, ..."
90,Height: 32 Width: 32,bird,"[1.71786928177, 0.0, 0.0,0.0, 0.0, 0.0, ...","[216.0, 195.0, 180.0,201.0, 178.0, 160.0, ..."
121,Height: 32 Width: 32,bird,"[0.0, 0.23753464222, 0.0,0.0, 0.0, 0.0, ...","[93.0, 96.0, 88.0, 102.0,106.0, 97.0, 117.0, ..."
138,Height: 32 Width: 32,bird,"[0.658935725689, 0.0,0.0, 0.0, 0.0, 0.0, ...","[205.0, 193.0, 195.0,200.0, 187.0, 193.0, ..."
335,Height: 32 Width: 32,bird,"[0.0, 0.0, 0.0, 0.0, 0.0,0.0, 8.50706672668, 0.0, ...","[160.0, 159.0, 154.0,162.0, 161.0, 156.0, ..."
560,Height: 32 Width: 32,bird,"[1.69159495831, 0.0, 0.0,0.0, 0.0, 0.0, ...","[147.0, 138.0, 88.0,151.0, 142.0, 92.0, ..."
649,Height: 32 Width: 32,bird,"[0.511156201363,0.324165046215, 0.0, ...","[65.0, 127.0, 9.0, 127.0,160.0, 15.0, 159.0, ..."
775,Height: 32 Width: 32,bird,"[0.0, 0.0, 0.0, 0.0, 0.0,0.0, 10.0127315521, 0.0, ...","[29.0, 41.0, 25.0, 29.0,42.0, 25.0, 28.0, 41.0, ..."
802,Height: 32 Width: 32,bird,"[0.277166724205, 0.0,0.0, 0.0, 0.0, 0.0, ...","[233.0, 230.0, 173.0,222.0, 218.0, 168.0, ..."
975,Height: 32 Width: 32,bird,"[0.0, 0.0336718559265,0.0, 0.645326733589, ...","[59.0, 180.0, 110.0,88.0, 186.0, 117.0, ..."


In [32]:
automobile_data=image_train_data[image_train_data['label']=='automobile']

In [33]:
automobile_data

id,image,label,deep_features,image_array
97,Height: 32 Width: 32,automobile,"[1.57818555832, 0.0, 0.0,0.0, 0.0, 0.0, ...","[33.0, 44.0, 27.0, 29.0,44.0, 31.0, 32.0, 45.0, ..."
136,Height: 32 Width: 32,automobile,"[0.0, 0.0, 0.0, 0.0, 0.0,0.0, 7.5737862587, 0.0, ...","[35.0, 59.0, 53.0, 36.0,56.0, 56.0, 42.0, 62.0, ..."
302,Height: 32 Width: 32,automobile,"[0.583938002586, 0.0,0.0, 0.0, 0.0, 0.0, ...","[64.0, 52.0, 37.0, 85.0,60.0, 40.0, 92.0, 66.0, ..."
312,Height: 32 Width: 32,automobile,"[0.0, 0.0, 0.0,0.392823398113, 0.0, ...","[124.0, 126.0, 113.0,124.0, 126.0, 113.0, ..."
323,Height: 32 Width: 32,automobile,"[0.0, 0.0, 0.0,4.42310428619, ...","[241.0, 241.0, 241.0,238.0, 238.0, 238.0, ..."
536,Height: 32 Width: 32,automobile,"[0.0, 0.0, 0.0, 0.0, 0.0,0.0, 8.42903900146, 0.0, ...","[164.0, 154.0, 154.0,128.0, 119.0, 120.0, ..."
593,Height: 32 Width: 32,automobile,"[1.65033948421, 0.0, 0.0,0.0, 0.0, 0.0, ...","[231.0, 222.0, 227.0,232.0, 217.0, 221.0, ..."
962,Height: 32 Width: 32,automobile,"[0.0, 0.0, 0.0,0.39552795887, 0.0, 0.0, ...","[255.0, 255.0, 255.0,255.0, 255.0, 255.0, ..."
997,Height: 32 Width: 32,automobile,"[0.0, 0.0, 0.0, 0.0, 0.0,0.0, 8.04085636139, 0.0, ...","[145.0, 148.0, 157.0,131.0, 134.0, 145.0, ..."
1421,Height: 32 Width: 32,automobile,"[0.0, 0.0, 0.0, 0.0, 0.0,0.359612941742, ...","[114.0, 95.0, 33.0,118.0, 98.0, 26.0, 91.0, ..."


## Model Building for each type ~ Nearest neighbors model

In [34]:
dog_model=graphlab.nearest_neighbors.create(dog_data,features=['deep_features'],label='id')

Starting brute force nearest neighbors model training.

In [35]:
cat_model=graphlab.nearest_neighbors.create(cat_data,features=['deep_features'],label='id')

Starting brute force nearest neighbors model training.

In [36]:
bird_model=graphlab.nearest_neighbors.create(bird_data,features=['deep_features'],label='id')

Starting brute force nearest neighbors model training.

In [37]:
automobile_model=graphlab.nearest_neighbors.create(automobile_data,features=['deep_features'],label='id')

Starting brute force nearest neighbors model training.

## Testing the trained knn models

In [38]:
show_dog_neighbors = lambda i: get_images_from_ids(dog_model.query(dog_data[i:i+1]))['image'].show()

In [39]:
show_cat_neighbors= lambda i: get_images_from_ids(cat_model.query(cat_data[i:i+1]))['image'].show()

In [40]:
show_bird_neighbors= lambda i: get_images_from_ids(bird_model.query(bird_data[i:i+1]))['image'].show()

In [41]:
show_automobile_neighbors= lambda i: get_images_from_ids(automobile_model.query(automobile_data[i:i+1]))['image'].show()

In [42]:
cat_data['image'].show()

In [43]:
image_test_data=graphlab.SFrame('image_test_data')

In [44]:
image_test_data['image'].show()

In [45]:
cat_model.query(image_test_data)

Starting blockwise querying.

max rows per data block: 4348

number of reference data blocks: 2

number of query data blocks: 1

+--------------+---------+-------------+--------------+

...

+--------------+---------+-------------+--------------+

| 4000         | 1020000 | 50.0982     | 2.33s        |

| Done         | 2036000 | 100         | 2.55s        |

+--------------+---------+-------------+--------------+

query_label,reference_label,distance,rank
0,16289,34.623719208,1
0,45646,36.0068799284,2
0,32139,36.5200813436,3
0,25713,36.7548502521,4
0,331,36.8731228168,5
1,46056,42.1485345986,1
1,44673,42.2954394873,2
1,11293,42.7881127497,3
1,1969,43.3088194975,4
1,14122,43.4878115357,5


In [46]:
get_images_from_ids(cat_model.query(image_test_data[0:1]))['image'].show()

Starting pairwise querying.

+--------------+---------+-------------+--------------+

| Query points | # Pairs | % Complete. | Elapsed Time |

+--------------+---------+-------------+--------------+

| 0            | 1       | 0.196464    | 37.676ms     |

| Done         |         | 100         | 269.13ms     |

+--------------+---------+-------------+--------------+

In [47]:
cat_model.query(image_test_data[0:1])

Starting pairwise querying.

+--------------+---------+-------------+--------------+

| Query points | # Pairs | % Complete. | Elapsed Time |

+--------------+---------+-------------+--------------+

| 0            | 1       | 0.196464    | 25.97ms      |

| Done         |         | 100         | 140.613ms    |

+--------------+---------+-------------+--------------+

query_label,reference_label,distance,rank
0,16289,34.623719208,1
0,45646,36.0068799284,2
0,32139,36.5200813436,3
0,25713,36.7548502521,4
0,331,36.8731228168,5


In [48]:
dog_model.query(image_test_data[0:1])

Starting pairwise querying.

+--------------+---------+-------------+--------------+

| Query points | # Pairs | % Complete. | Elapsed Time |

+--------------+---------+-------------+--------------+

| 0            | 1       | 0.196464    | 55.526ms     |

| Done         |         | 100         | 231.742ms    |

+--------------+---------+-------------+--------------+

query_label,reference_label,distance,rank
0,16976,37.4642628784,1
0,13387,37.5666832169,2
0,35867,37.6047267079,3
0,44603,37.7065585153,4
0,6094,38.5113254907,5


In [49]:
get_images_from_ids(dog_model.query(image_test_data[0:1]))['image'].show()

Starting pairwise querying.

+--------------+---------+-------------+--------------+

| Query points | # Pairs | % Complete. | Elapsed Time |

+--------------+---------+-------------+--------------+

| 0            | 1       | 0.196464    | 60.253ms     |

| Done         |         | 100         | 210.313ms    |

+--------------+---------+-------------+--------------+

## Task -3

In [50]:
cat_model.query(image_test_data[0:1])['distance'].mean()

Starting pairwise querying.

+--------------+---------+-------------+--------------+

| Query points | # Pairs | % Complete. | Elapsed Time |

+--------------+---------+-------------+--------------+

| 0            | 1       | 0.196464    | 20.112ms     |

| Done         |         | 100         | 122.537ms    |

+--------------+---------+-------------+--------------+

36.15573070978294

In [51]:
dog_model.query(image_test_data[0:1])['distance'].mean()

Starting pairwise querying.

+--------------+---------+-------------+--------------+

| Query points | # Pairs | % Complete. | Elapsed Time |

+--------------+---------+-------------+--------------+

| 0            | 1       | 0.196464    | 23.681ms     |

| Done         |         | 100         | 165.63ms     |

+--------------+---------+-------------+--------------+

37.77071136184157

## Split the test data into specifics


In [52]:
image_test_cat=image_test_data[image_test_data['label']=='cat']

In [53]:
image_test_dog=image_test_data[image_test_data['label']=='dog']

In [54]:
image_test_bird=image_test_data[image_test_data['label']=='bird']

In [55]:
image_test_automobile=image_test_data[image_test_data['label']=='automobile']

## Dog queries

In [56]:
dog_cat_neighbors= cat_model.query(image_test_dog,k=1)

Starting blockwise querying.

max rows per data block: 4348

number of reference data blocks: 2

number of query data blocks: 1

+--------------+---------+-------------+--------------+

| Query points | # Pairs | % Complete. | Elapsed Time |

+--------------+---------+-------------+--------------+

| 1000         | 254000  | 49.9018     | 847.835ms    |

| Done         | 509000  | 100         | 999.577ms    |

+--------------+---------+-------------+--------------+

In [57]:
dog_cat_neighbors

query_label,reference_label,distance,rank
0,33,36.4196077068,1
1,30606,38.8353268874,1
2,5545,36.9763410854,1
3,19631,34.5750072914,1
4,7493,34.778824791,1
5,47044,35.1171578292,1
6,13918,40.6095830913,1
7,10981,39.9036867306,1
8,45456,38.0674700168,1
9,44673,42.7258732951,1


In [58]:
dog_automobile_neighbors=automobile_model.query(image_test_dog,k=1)
dog_automobile_neighbors

Starting blockwise querying.

max rows per data block: 4348

number of reference data blocks: 2

number of query data blocks: 1

+--------------+---------+-------------+--------------+

...

+--------------+---------+-------------+--------------+

| 1000         | 255000  | 50.0982     | 1.13s        |

| Done         | 509000  | 100         | 1.14s        |

+--------------+---------+-------------+--------------+

query_label,reference_label,distance,rank
0,33859,41.9579761457,1
1,2046,46.0021331807,1
2,19594,42.9462290692,1
3,11000,41.6866060048,1
4,19594,39.2269664935,1
5,49314,40.5845117698,1
6,40822,45.1067352961,1
7,44997,41.3221140974,1
8,33859,41.8244654995,1
9,33859,45.4976929401,1


In [59]:
dog_bird_neighbors=bird_model.query(image_test_dog,k=1)
dog_bird_neighbors

Starting blockwise querying.

max rows per data block: 4348

number of reference data blocks: 2

number of query data blocks: 1

+--------------+---------+-------------+--------------+

...

+--------------+---------+-------------+--------------+

| 1000         | 239000  | 50          | 631.951ms    |

| Done         | 478000  | 100         | 687.523ms    |

+--------------+---------+-------------+--------------+

query_label,reference_label,distance,rank
0,44658,41.7538647304,1
1,9215,41.3382958925,1
2,36675,38.6157590853,1
3,12582,37.0892269954,1
4,36122,38.272288694,1
5,8736,39.1462089236,1
6,38991,40.523040106,1
7,44177,38.1947918393,1
8,4549,40.1567131661,1
9,40225,45.5597962603,1


In [60]:
dog_dog_neighbors= dog_model.query(image_test_dog,k=1)
dog_dog_neighbors

Starting blockwise querying.

max rows per data block: 4348

number of reference data blocks: 2

number of query data blocks: 1

+--------------+---------+-------------+--------------+

...

+--------------+---------+-------------+--------------+

| 1000         | 255000  | 50.0982     | 1.06s        |

| Done         | 509000  | 100         | 1.11s        |

+--------------+---------+-------------+--------------+

query_label,reference_label,distance,rank
0,49803,33.4773590373,1
1,5755,32.8458495684,1
2,20715,35.0397073189,1
3,13387,33.9010327697,1
4,12089,37.4849250909,1
5,6094,34.945165344,1
6,3431,39.0957278345,1
7,6184,37.7696131032,1
8,2167,35.1089144603,1
9,7776,43.2422832585,1


In [61]:
dog_distances=graphlab.SFrame()
dog_distances['dog-cat']=dog_cat_neighbors['distance']
dog_distances['dog-bird']=dog_bird_neighbors['distance']
dog_distances['dog-automobile']=dog_automobile_neighbors['distance']
dog_distances['dog-dog']=dog_dog_neighbors['distance']

In [62]:
dog_distances

dog-cat,dog-bird,dog-automobile,dog-dog
36.4196077068,41.7538647304,41.9579761457,33.4773590373
38.8353268874,41.3382958925,46.0021331807,32.8458495684
36.9763410854,38.6157590853,42.9462290692,35.0397073189
34.5750072914,37.0892269954,41.6866060048,33.9010327697
34.778824791,38.272288694,39.2269664935,37.4849250909
35.1171578292,39.1462089236,40.5845117698,34.945165344
40.6095830913,40.523040106,45.1067352961,39.0957278345
39.9036867306,38.1947918393,41.3221140974,37.7696131032
38.0674700168,40.1567131661,41.8244654995,35.1089144603
42.7258732951,45.5597962603,45.4976929401,43.2422832585


In [97]:
count=0
def is_dog_correct(row):
    mini=min(row['dog-cat'],row['dog-bird'],row['dog-automobile'],row['dog-dog'])
    if row['dog-dog']==mini:
        global count
        count+=1
    
    

In [98]:
dog_distances.apply(is_dog_correct)

dtype: float
Rows: 1000
[None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, ... ]

In [104]:
dog_distances

dog-cat,dog-bird,dog-automobile,dog-dog
36.4196077068,41.7538647304,41.9579761457,33.4773590373
38.8353268874,41.3382958925,46.0021331807,32.8458495684
36.9763410854,38.6157590853,42.9462290692,35.0397073189
34.5750072914,37.0892269954,41.6866060048,33.9010327697
34.778824791,38.272288694,39.2269664935,37.4849250909
35.1171578292,39.1462089236,40.5845117698,34.945165344
40.6095830913,40.523040106,45.1067352961,39.0957278345
39.9036867306,38.1947918393,41.3221140974,37.7696131032
38.0674700168,40.1567131661,41.8244654995,35.1089144603
42.7258732951,45.5597962603,45.4976929401,43.2422832585


In [103]:
dog_distances['result']

In [105]:
get_images_from_ids(knn_model.query(image_test_data[0:1]))['image'].show()

Starting pairwise querying.

+--------------+---------+-------------+--------------+

| Query points | # Pairs | % Complete. | Elapsed Time |

+--------------+---------+-------------+--------------+

| 0            | 1       | 0.0498753   | 26.545ms     |

| Done         |         | 100         | 415.487ms    |

+--------------+---------+-------------+--------------+

In [108]:
dog_model.summary()

Class                          : NearestNeighborsModel

Attributes
----------
Method                         : brute_force
Number of distance components  : 1
Number of examples             : 509
Number of feature columns      : 1
Number of unpacked features    : 4096
Total training time (seconds)  : 0.3113



In [110]:
graphlab.compare(image_test_data,[dog_model,cat_model,automobile_model,bird_model]).show()